In [ ]:
import sys
sys.path.append("..")
from figutils import *
from string import ascii_lowercase
import matplotlib.patheffects as pe

In [ ]:
output_dir = os.path.join(OUTPUT_DIR, os.path.basename(os.getcwd()))

In [ ]:
methods = ["baseline", "debiased"]
method_names = ["Naive", "Debiased"]
losses = []
imgs_final = []
imgs = []

ref_img = mi.Bitmap(os.path.join(output_dir, "img_ref.exr"))
for method in methods:
    losses.append(np.load(os.path.join(output_dir, method, "loss_hist.npz")))
    imgs.append(mi.TensorXf(mi.Bitmap(os.path.join(output_dir, method, "img", "0249.exr"))))
    imgs_final.append(mi.TensorXf(mi.Bitmap(os.path.join(output_dir, method, "img_inf", "0249.exr"))))


In [ ]:
n_cols = 3
n_rows = 2
base_size = 3
h,w,_ = imgs[0].shape

wspace = 0.02
hspace = 0.02 * w/h
aspect = gridspec_aspect(n_rows, n_cols, w, h, wspace=wspace, hspace=hspace)

sns.set_style('white')
fig = plt.figure(1, figsize=(TEXT_WIDTH, TEXT_WIDTH / aspect))
gs = fig.add_gridspec(n_rows, n_cols, wspace=wspace, hspace=hspace)

for i, method in enumerate(method_names):
    ax = fig.add_subplot(gs[0, i])
    ax.imshow(mi.util.convert_to_bitmap(imgs[i]), interpolation='none')
    std = dr.sqrt(dr.mean(dr.sqr(imgs[i] - imgs_final[i])))[0]
    text = f"$\sigma={std:.3f}$"
    ax.text(0.99*w, 0.99*h, text, ha="right", va="bottom", color="white", fontsize=DEFAULT_FONTSIZE, path_effects=[pe.withStroke(linewidth=1, foreground="black")])
    if i == 0:
        ax.set_ylabel("Primal")
    disable_ticks(ax)

    ax = fig.add_subplot(gs[1, i])
    ax.imshow(mi.util.convert_to_bitmap(imgs_final[i]), interpolation='none')
    if i == 0:
        ax.set_ylabel("Re-rendered")
    disable_ticks(ax)
    ax.set_title(rf"\textbf{{({ascii_lowercase[i]})}} {method}", y=-0.3)

ax = fig.add_subplot(gs[1, 2])
ax.imshow(mi.util.convert_to_bitmap(ref_img), interpolation='none')
disable_ticks(ax)
ax.set_title(r"\textbf{(c)} Target", y=-0.3)

save_fig('debiasing_v2')